# Perform Research with Multi-Agent Group Chat

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

## Requirements

````{=mdx}
:::info Requirements
Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

In [1]:
from setup_environment_r2 import set_environment_variables
set_environment_variables('GroupChat_Research')
import os
import autogen

import agentops
agentops.init(os.environ.get("AGENTSOPS_API_KEY"))

reading environment variables from: </home/tjamil/Desktop/Agents>
API Keys loaded and tracing set with project name:  GroupChat_Research


🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=97e975aa-2650-43c6-8fcc-661a34c83682


## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [2]:
import os
llm_config={
    "config_list" : [{
        "api_type": "groq",
        "model": "llama-3.1-70b-versatile",
        "api_key": os.environ.get("GROQ_API_KEY") 
        }]
    }

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

## Construct Agents

In [3]:
llm_configx = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": llm_config,
    "timeout": 120,
}

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)

scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=llm_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code.""",
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
""",
    llm_config=llm_config,
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=llm_config,
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor, critic], messages=[], max_round=50
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

## Start Chat

In [4]:
user_proxy.initiate_chat(
    manager,
    message="""
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
""",
)

Admin (to chat_manager):


find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.


--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

**Initial Plan:**

**Title:** Weekly LLM Applications Update from arXiv

**Objective:** To curate a list of recent papers on Large Language Model (LLM) applications from arXiv and create a markdown table categorizing them by domain.

**Step-by-Step Plan:**

1. **Scientist (Researcher):** Perform a search on arXiv for papers related to LLM applications published in the last week (7 days).
	* Use relevant keywords like "LLM," "transformer," "natural language processing," "language model," etc.
	* Filter the search results by date to get papers from the last week.
2. **Engineer:** Write a Python script to retrieve the search results from arXiv and extract the paper titles, authors, dates, and abstracts.
	* Utilize libraries 

/home/tjamil/.local/share/virtualenvs/Agents-4yRd6dJo/lib/python3.11/site-packages/autogen/oai/groq.py:286: UserWarning: Cost calculation not available for model llama-3.1-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Critic (to chat_manager):

Your code is well-structured and does what the initial plan outlined. Here are a few minor suggestions to improve readability, maintainability, and robustness:

**Improvements:**

1.  **Error Handling:** Currently, the code assumes that the `arxiv.query()` function will always return results. However, this may not always be the case, especially if there are issues with the network connection or the query itself. Consider adding error handling code to handle such situations.

2.  **Abstract Length Limitation:** You're currently limiting the abstract length to 200 characters using `abstract[:200]`. This can result in truncated abstracts that might not convey the full meaning. You might want to consider increasing the character limit or even better, dynamically adjusting the limit based on the available space in the markdown table.

3.  **Debugging Statements:** It's always helpful to have print statements in place for debugging purposes. You can use a library l

🖇 AgentOps: Could not post events - RequestException: HTTPSConnectionPool(host='api.agentops.ai', port=443): Max retries exceeded with url: /v2/create_events (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f09935cb190>: Failed to resolve 'api.agentops.ai' ([Errno -3] Temporary failure in name resolution)"))


## Create Group Chat without Critic for Comparison

In [ ]:
groupchat_nocritic = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor], messages=[], max_round=50
)

for agent in groupchat.agents:
    agent.reset()

manager_nocritic = autogen.GroupChatManager(groupchat=groupchat_nocritic, llm_config=gpt4_config)

user_proxy.initiate_chat(
    manager_nocritic,
    message="""
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
""",
)